In [2]:
import pandas as pd
import numpy as np
import pickle
from geopy.geocoders import Nominatim
from haversine import haversine, Unit

In [3]:
# Load dataset
data = pd.read_csv("Attraction.csv")

In [ ]:
# Load saved models/matrices
hybrid_matrix = np.load("models/hybrid_matrix.npy")
with open("models/knn_model_hybrid.pkl", "rb") as f:
    knn_model_hybrid = pickle.load(f)


In [4]:
knn_model_hybrid

NearestNeighbors(metric='precomputed', n_neighbors=4)

In [6]:
# Initialize geolocator
geolocator = Nominatim(user_agent="NepalLocationApp")

In [7]:
def get_latitude_longitude(location_name):
    location = geolocator.geocode(location_name + ", Nepal")
    if location:
        return location.latitude, location.longitude
    else:
        print(f"Could not find the location '{location_name}'. Please check the name.")
        return None, None

In [8]:
get_latitude_longitude('balkhu')

(27.6849927, 85.29786)

In [19]:
def recommend_based_on_tags(user_tag, data, k=35):
    return data[data['tag'] == user_tag][['Destination']].head(k)

In [20]:
def recommend_location_by_raw_proximity(user_lat, user_lon, data, k=5):
    distances = []
    for _, row in data.iterrows():
        distance = haversine((user_lat, user_lon), (row['latitude'], row['longitude']), unit=Unit.KILOMETERS)
        distances.append(distance)
    data['distance'] = distances
    return data.sort_values(by='distance').head(k)[['Destination']]

In [21]:
def combine_similarity_and_proximity(sim_matrix, prox_matrix, alpha=0.7):
    return alpha * sim_matrix + (1 - alpha) * prox_matrix

In [23]:
user_location_name = input(" Enter your current location or nearest visited destination: ")
location_in_dataset = data[data['Destination'].str.contains(user_location_name, case=False, na=False)]

if not location_in_dataset.empty:
    # print(f" Location found in dataset: {location_in_dataset['Destination'].values[0]}")
    user_index = location_in_dataset.index[0]

    user_tag = data.loc[user_index, 'tag']
    user_lat = data.loc[user_index, 'latitude']
    user_lon = data.loc[user_index, 'longitude']

    print("Similar Places to visit:")
    print(recommend_based_on_tags(user_tag, data, k=5))

    print("\n Nearest destination:")
    print(recommend_location_by_raw_proximity(user_lat, user_lon, data, k=5))

 # Hybrid recommendation (requires cosine_sim_matrix and proximity_matrix)
    try:
        cosine_sim_matrix = np.load("models/cosine_sim_matrix.npy")
        proximity_matrix = np.load("models/proximity_matrix.npy")

        print("\n Popular destination you may like : ")
        combined_matrix = combine_similarity_and_proximity(cosine_sim_matrix, proximity_matrix, alpha=0.7)

        distances, indices = knn_model_hybrid.kneighbors([combined_matrix[user_index]], n_neighbors=5)
        recommended_locations_hybrid = data.loc[indices[0][1:], ['Destination']]
        print(recommended_locations_hybrid)
    except Exception as e:
        print("⚠️ Hybrid recommendation could not be computed:", str(e))

else:
    #print(f"⚠️ Location '{user_location_name}' not found in dataset.")
    lat, lon = get_latitude_longitude(user_location_name)

    if lat is not None and lon is not None:
        print(f"\n Using coordinates: {lat}, {lon}")
        print("Nearest destination :  ")
        print(recommend_location_by_raw_proximity(lat, lon, data, k=5))
    else:
        print("❌ Cannot provide recommendations due to location error.")


 Enter your current location or nearest visited destination:  taudaha


Similar Places to visit:
         Destination
50      Rani Pokhari
51      Naag Pokhari
52     Kamal Pokhari
53  Pimbahal Pokhari
54    Siddha Pokhari

 Nearest destination:
                              Destination
55                           Taudaha Lake
61  Whoopee Land Amusement and Water Park
75                        Manjushree Park
77                        Champadevi Hike
49                   Karya Binayak Temple

 Popular destination you may like : 
             Destination
75       Manjushree Park
55          Taudaha Lake
77       Champadevi Hike
49  Karya Binayak Temple
